# 要約 
このJupyter Notebookは、「20の質問」ゲームに取り組むための言語モデルを実装するための準備を行っています。具体的には、MicrosoftのPhi-3 Miniモデルを使用して、質問を行い、その回答を適切に処理するエージェントを構築しています。

### 問題への取り組み
ノートブックの目的は、Kaggleの「LLM 20 Questions」コンペティションに参加するためのモデルを設定し、特に質問者（guesser）と回答者（answerer）の役割を制御するエージェントを作成することです。このエージェントは、ユーザーが思い浮かべる特定の「ターゲットワード」を答えるために、制約のある「はい」または「いいえ」の質問を投げかける必要があります。

### 手法とライブラリ
1. **パッケージのインストール**:
   - `tqdm`: 進捗バーを表示するためのライブラリ。
   - `pydantic`: データバリデーションおよび設定管理に使用。
   - `transformers`: 言語モデルの操作、特にPhi-3モデルの読み込みと使用に利用。

2. **モデルのセットアップ**:
   - Hugging FaceからPhi-3モデルをダウンロードし、適切なディレクトリに保存します。
   - `AutoModelForCausalLM`と`AutoTokenizer`を使用して、言語モデルのロードを行い、テキスト生成のためのパイプラインを設定します。

3. **エージェントの実装**:
   - `play`関数が定義されており、KaggleObservationとKaggleConfigのデータクラスを通じて、ゲームの状態を管理します。
   - 質問者と回答者の役割に応じて異なるプロンプトを生成してモデルに問いかけることで、ゲームのロジックを実装しています。

4. **提出ファイルの準備**:
   - 最終的に、作成したコードを`main.py`というファイルに書き出し、提出用のディレクトリをtar.gz形式で圧縮する手順が含まれています。

### 結論
このノートブックは、明示的に設定されているライブラリと手順を用いて、20の質問ゲームにおけるAIエージェントの初期設定を行っており、将来的には実際のプレイに向けたモデルの精度を向上させるための基盤を築いています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に関連する初心者がつまずく可能性のある専門用語の解説を示します。これらは一般的に知られているものではないか、特定の文脈での理解が必要なものです。

1. **tqdm**: Pythonでのプログレスバーを簡単に実装するためのライブラリです。ループ処理中に進捗状況を視覚的に表示することができ、処理の進行具合を確認できるのが特徴です。

2. **pydantic**: Pythonのデータバリデーションおよび設定管理のためのライブラリで、データモデルを簡単に作成し、型検査を行うことができるため、データの整合性を保つのに役立ちます。

3. **Attributes**: Pythonのクラスにおけるプロパティ。通常、クラスのインスタンスが持つ情報を定義します。例えば、dataclassを使うと、属性の定義を簡単に行えます。

4. **pipeline**: Hugging FaceのTransformersライブラリの一部で、特定のタスクに対するモデルの推論を簡単に実行できるラッパーです。たとえば、テキスト生成、質問応答、分類などのタスクを簡単に処理できます。

5. **AutoModelForCausalLM**: 自然言語処理モデルの一種で、因果言語モデリングタスク（次の単語を予測するタスク）のためのモデルを自動で選択し、インスタンス化するためのクラスです。

6. **torch_dtype**: PyTorchにおけるデータ型を指定するための引数で、モデルの訓練や推論の際に使用されるデータ形式を設定します。例えば、FP16（16ビット浮動小数点数）やFP32（32ビット浮動小数点数）などがあります。

7. **temperature**: 確率的生成モデルにおけるサンプリングの制約を調整するパラメータです。値が高いと多様な生成結果が可能になり、低いと決定的な生成になります。

8. **yes-or-no question**: はい/いいえで答えられる質問。20の質問ゲームでは、情報を効率的に収集するためにこの形の質問が多用されます。

9. **dataclass**: Pythonのクラスを簡素化するための構文で、イミュータブルなデータ構造を作成する際に有用です。属性を定義するだけで多くのボイラープレートコードを省略できます。

10. **CUDA**: NVIDIAの並列計算プラットフォームおよびAPIで、GPUを用いて計算を加速する技術です。特にディープラーニングの訓練において、計算の時間を短縮するために頻繁に使用されます。

11. **model.gguf**: モデルの保存形式の一つで、特定のフレームワークに依存せずにモデルデータを格納するために設計されています。

12. **Kaggle Observation / Config**: このコンペティションでのゲームの状態や設定を保持するために使用されるデータ構造。各エピソードの進行状況や環境に関する情報を格納します。

13. **play関数**: ゲームを実行するための主要な関数であり、観察データと設定に基づき、適切な行動を決定し、モデルを通じて結果を生成します。

これらの用語は、特にJupyter Notebookの特定のコンテキストや実装において、理解に重要であることが多いです。

---


# Phi-3との20の質問

## 提出フォルダにパッケージをインストールする

In [ ]:
# 必要なパッケージをインストールします。
# - tqdm: 進捗バーを表示するためのライブラリ
# - pydantic: データバリデーションおよび設定管理用ライブラリ
# - transformers: 言語モデルを扱うためのライブラリ
# 提出フォルダにインストールし、他の環境に影響を与えないようにします。

pip install -U -t /kaggle/working/submission/lib tqdm pydantic transformers -qq  # 提出フォルダにパッケージをインストールします。

## Llama.cppのセットアップ（現時点では未使用）

In [ ]:
# Llama.cpp用のPythonパッケージをインストールするためのコマンドです。
# CMAKE_ARGSを設定して、CUDAを使用するように指定しています。
# このパッケージは、現在使用されていませんが、将来的に必要になるかもしれません。
# 指定されたディレクトリにインストールします。

# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install -t /kaggle/working/submission/lib llama-cpp-python \
#   --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121

In [ ]:
# 必要なディレクトリを作成します。
# -pオプションを使うことで、親ディレクトリが存在しない場合も一緒に作成されます。
# ここでは、提出フォルダ内にphi3というサブディレクトリを作成します。

mkdir -p /kaggle/working/submission/lib/phi3/

In [ ]:
# Hugging FaceからPhi-3モデルをダウンロードするためのコマンドです。
# curlを使用して指定されたURLからファイルを取得し、
# その内容を提出フォルダのphi3ディレクトリに"model.gguf"として保存します。

# !curl -L "https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf?download=true" > "/kaggle/working/submission/lib/phi3/model.gguf"

In [ ]:
# 必要なモジュールをインポートします。
# osとsysを使用して、環境に応じた重みファイルのパスを設定しています。

# KAGGLE_AGENT_PATHはKaggleのエージェントのパスです。
# もしこのパスが存在する場合、libディレクトリをPythonのモジュール検索パスに追加します。
# その後、重みファイルのパスをKAGGLE_AGENT_PATH内のモデルファイルに設定します。
# そのパスが存在しない場合は、提出フォルダのlibディレクトリを検索パスに追加し、重みファイルのパスをそこに設定します。

import os, sys
KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    sys.path.insert(0, os.path.join(KAGGLE_AGENT_PATH, 'lib'))
    WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "lib/phi3/model.gguf")
else:
    sys.path.insert(0, "/kaggle/working/submission/lib")
    WEIGHTS_PATH = "/kaggle/working/submission/lib/phi3/model.gguf"

In [ ]:
# 現在の作業ディレクトリ内のファイルとフォルダのリストを表示します。
# %lsコマンドを使用することで、現在の環境に保存されているすべてのファイルとディレクトリを確認できます。

%ls

## 提出ファイルの作成

In [ ]:
%%writefile submission/main.py
# 提出ファイルであるmain.pyを作成します。
from pydantic.dataclasses import dataclass
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from typing import Literal, List
import os, sys, json

KAGGLE_AGENT_PATH = "/kaggle_simulations/agent/"
if os.path.exists(KAGGLE_AGENT_PATH):
    os.chdir(os.path.join(KAGGLE_AGENT_PATH, 'lib'))
    #WEIGHTS_PATH = os.path.join(KAGGLE_AGENT_PATH, "lib/phi3/model.gguf")
else:
    os.chdir("/kaggle/working/submission/lib")
    #WEIGHTS_PATH = "/kaggle/working/submission/lib/phi3/model.gguf"
    
print(f"Current Directory is {os.getcwd()}. \nFiles in here: {', '.join(os.listdir())}")

# モデルをインポートします。

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",  
    device_map="cuda", torch_dtype="auto", trust_remote_code=True,
    cache_dir="./huggingface"
    
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", cache_dir="./huggingface")
hf_llm = pipeline("text-generation", model=model, tokenizer=tokenizer)

def ask(prompt: str, max_new_tokens=100) -> str:
    result = hf_llm(text_inputs=prompt, return_full_text=False, temperature=0.2, do_sample=False, max_new_tokens=max_new_tokens)
    return result[0]['generated_text']

assert ask("<|user|>\nHello!<|end|>\n<|assistant|>")

@dataclass
class KaggleObservation:
  remainingOverageTime: int | float
  step: int

  questions: list[str]
  answers: list[str]
  guesses: list[str]

  role: Literal["guesser", "answerer"]
  turnType: Literal["ask", "guess", "answer"]

  keyword: str
  category: str

@dataclass
class KaggleConfig:
  episodeSteps: int
  actTimeout: int | float
  runTimeout: int | float
  agentTimeout: int | float
  __raw_path__: str

# llm = Llama(
#   model_path=WEIGHTS_PATH,  # GGUFファイルへのパス
#   n_ctx=2048,  # 使用する最大シーケンス長 - シーケンスが長くなるほど多くのリソースが必要になります
#   n_threads=4, # 使用するCPUスレッド数。システムや性能に応じて調整します
#   n_gpu_layers=35, # GPUにオフロードするレイヤーの数。GPUアクセラレーションが使用できない場合は0に設定します。
#   use_mlock=True, # RAMにメモリをロックし、ディスクにスワップされないようにするかどうか。大きなモデルには有用です
#   use_mmap=False, #
# )

def get_context_prompt(observation: KaggleObservation) -> str:
  questions = observation.questions
  answers = observation.answers

  history_prompt = ""
  for index in range(len(max(questions, answers))):
    history_prompt += f"<|user|>\n{questions[index]}<|end|>\n" if index < len(questions) else ""
    history_prompt += f"<|assistant|>\n{answers[index]}<|end|>\n" if index < len(answers) else ""
  #history_prompt += "<|assistant|>\n"
  
  return history_prompt

def get_guesser_prompt(observation: KaggleObservation) -> str:
  prompt = f"<|user|>\nLet's play 20 Questions. You are playing the role of the {observation.role.title()}.<|end|>\n"
  prompt += get_context_prompt(observation)

  if observation.turnType == "ask":
    prompt += f"<|user|>\nTake a break, and ask a short yes-or-no question that would be useful to determine what the city I'm thinking about. Previous questions have been listed above. KEEP YOUR QUESTION ONE SENTENCE ONLY! Do not add any explaination to why you chose the question.<|end|>\n"
  elif observation.turnType == "guess":
    prompt += f"<|user|>\nNow, based on the information above, guess what city I'm thinking about, " 
    prompt += f"which aren't these: {', '.join(observation.guesses)}."
    prompt += f"Now, Make an informed guess, and only provide one word!<|end|>\n"
  else:
    raise ValueError(f"Invalid turnType: {observation.turnType}\n\n{observation}")
  
  prompt += "<|assistant|>\n"

  return prompt

def get_answerer_prompt(observation: KaggleObservation) -> str:
  prompt = f"<|user|>\nYou are a highly experienced tour guide specialized in the city {', '.join(observation.keyword.split(' '))}.\n"
  prompt += "You must answer a question about this city accurately, but only using the word **yes** or **no**.<|end|>\n"

  prompt += f"<|user|>{observation.questions[-1]}<|end|>\n"
  prompt += "<|assistant|>\n"
  return prompt


def play(obs, conf):
  print("Observation: " + json.dumps(obs, indent=2, ensure_ascii=False))
  print("Confing: " + json.dumps(conf, indent=2, ensure_ascii=False))
  observation = KaggleObservation(**obs)
  config = KaggleConfig(**conf)
  if observation.role == "guesser":
    prompt = get_guesser_prompt(observation)
    result = ask(prompt, max_new_tokens=40).split("\n")[0].strip()#, stop=["<|end|>"], max_tokens=256, temperature=0.5, echo=False)
  elif observation.role == "answerer":
    prompt = get_answerer_prompt(observation)
    answer = ask(prompt, max_new_tokens=20)#, stop=["<|end|>"], max_tokens=20, temperature=0.5, echo=False)
    result = "no" if "no" in answer else "yes"
  else:
    raise ValueError(f"Invalid role: {observation.role}\n\n{observation}")
  print(f"Result: {result}")
  return result

## ちょっと確認中

In [ ]:
# 提出ファイルのmain.pyからすべての定義をインポートします。
# これにより、main.pyの関数やクラスをこのセル内で使用できるようにします。

from submission.main import *

In [ ]:
# play関数を呼び出して、仮のKaggleObservationおよびKaggleConfigオブジェクトを作成します。
# assert文を使用して、play関数が正しく動作するか確認します。
# ここでは、'guesser'のロールで質問をするターンを模擬しており、まだ質問や推測は行われていません。

assert play({
  'remainingOverageTime': 300, 
  'step': 0, 
  'questions': [], 
  'guesses': [], 
  'answers': [], 
  'role': 'guesser', 
  'turnType': 'ask', 
  'keyword': '', # 例: bangkok
  'category': '', # 例: city
}, {
  'episodeSteps': 61, 
  'actTimeout': 60, 
  'runTimeout': 9600, 
  'agentTimeout': 3600, 
  '__raw_path__': '/kaggle_simulations/agent/main.py'
})

In [ ]:
# play関数を呼び出し、今回は'answerer'のロールで、特定の質問に対して回答を返すターンを模擬します。
# アサーションを使用して、play関数が正しく機能するかを確認します。
# 質問として、「考えている都市は北アメリカにありますか？」が与えられています。

assert play({
  'remainingOverageTime': 300, 
  'step': 0, 
  'questions': ["Is the city you're thinking of located in North America?"], 
  'guesses': [], 
  'answers': [], 
  'role': 'answerer', 
  'turnType': 'answer', 
  'keyword': '', # 例: bangkok
  'category': '', # 例: city
}, {
  'episodeSteps': 61, 
  'actTimeout': 60, 
  'runTimeout': 9600, 
  'agentTimeout': 3600, 
  '__raw_path__': '/kaggle_simulations/agent/main.py'
})

## 提出するためにディレクトリをtar.gz形式でアーカイブする

In [ ]:
# tar.gzファイルを圧縮するためにpigzとpvというツールをインストールします。
# インストールの出力は表示しないように/dev/nullにリダイレクトしています。

!apt install pigz pv > /dev/null

In [ ]:
# 作業ディレクトリを/kaggle/working/に移動します。
# ここでアーカイブを作成する準備を行います。

%cd /kaggle/working/

In [ ]:
# submissionディレクトリをtar.gz形式でアーカイブします。
# pigzを使用して圧縮し、pvで進行状況を表示します。
# -Cオプションは指定したディレクトリに移動してからアーカイブを作成するためのものです。
# このコマンドでは、submission.tar.gzというファイルが作成されます。

!tar --use-compress-program='pigz --fast --recursive | pv' -cf submission.tar.gz -C /kaggle/working/submission .

In [ ]:
# アーカイブの作成が成功したことを示すメッセージを出力します。

print("Success.")